In [279]:
import numpy as np
from sklearn.utils.extmath import fast_logdet
from sklearn.utils import check_random_state
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import empirical_covariance
from functools import partial

from network_inference.prox import prox_logdet, soft_thresholding_sign
from network_inference.datasets import is_pos_def
from network_inference.utils import _scalar_product, update_rho, convergence

In [246]:
def objective_H(H, R=None, T=None, K=None, U= None,_rho=1, _mu=1):
    return _rho* np.linalg.norm(R - T + U+ K.T.dot(np.linalg.inv(H).dot(K)))**2 + _mu*np.linalg.norm(H, 1)



    # return x.ravel().dot(y.ravel())

In [247]:
def _choose_lambda(lamda, R, T, K, H, U, _rho, _mu, prox, grad, gamma, delta=1e-4, eps=0.5, max_iter=1000):
    """Choose lambda for backtracking.

    References
    ----------
    Salzo S. (2017). https://doi.org/10.1137/16M1073741

    """
    partial_f = partial(objective_H, R=R, T=T, K=K, U=U,_rho=_rho, _mu=_mu)
    fx = partial_f(H)

    y_minus_x = prox - H
    tolerance = _scalar_product(y_minus_x, grad)
    tolerance += delta / gamma * _scalar_product(y_minus_x, y_minus_x)
#     print("Tolerance:", tolerance)
    for i in range(max_iter):
        # line-search
        x1 = H + lamda * y_minus_x

        loss_diff = partial_f(x1) - fx
        #print("Loss diff:", loss_diff)
        if loss_diff <= lamda * tolerance and is_pos_def(x1):
              break
        lamda *= eps
    else:
        print("did not find lambda")
    return lamda, i + 1

In [266]:
def _choose_gamma(gamma, H, R, T, K, U, _rho, _mu, _lambda, grad,
                 eps=0.5, max_iter=1000):
    """Choose gamma for backtracking.

    References
    ----------
    Salzo S. (2017). https://doi.org/10.1137/16M1073741

    """
    partial_f = partial(objective_H, R=R, T=T, K=K, U=U, _rho=_rho, _mu=_mu)
    fx = partial_f(H)
    for i in range(max_iter):
        prox = soft_thresholding_sign(H - gamma * grad, _mu * gamma)
        if positive_definite(prox):
            break
        gamma *= eps

    return gamma, prox

In [267]:
def _upgrade_H(R, T, K, U, _rho, _mu, verbose=0):
    H = np.random.rand(K.shape[0], K.shape[0])
    H = H.T.dot(H)
    _lambda = 1
    gamma = 1
    for iter_ in range(1000):
        H_old = H
        Hinv = np.linalg.inv(H)
        gradient = -_rho* K.dot(R - T + U + np.linalg.multi_dot((K.T, Hinv, K))).dot(K.T).dot(Hinv).dot(Hinv)
        gamma, _ = _choose_gamma(gamma, H, R, T, K, U, _rho,_mu, _lambda, gradient)
        Y = soft_thresholding_sign(H - gamma*gradient, _mu)
        _lambda,_ = _choose_lambda(_lambda, R, T, K, H, U,_rho, _mu, Y, gradient, 1, max_iter=1000, delta=1e-2)
      
        H = H + _lambda*(Y - H)
        obj = objective_H(H, R, T, K, U,_rho=_rho, _mu=_mu)
        iter_diff =np.linalg.norm(H - H_old) 
        if verbose:
            print("Iter: %d, obj: %.5f, iter_diff: %.5f"%(iter_, obj, iter_diff))
        if(iter_diff<1e-5):
            
            break
    else:
        print("Did not converge")
    return H

In [268]:
# R = np.random.rand(10,10)
# T = np.random.rand(10,10)
# U = np.random.rand(10,10)
# K = per_cov
# print(R.shape, T.shape, K.shape)
# _upgrade_H(R,T,K,1,0.02)

In [269]:
def objective(emp_cov, K, R, T, H, mu, eta, rho):
    res = fast_logdet(R)
    res += np.sum(R*emp_cov)
    res += rho/2 * np.linalg.norm(R - T + U + K.T.dot(np.linalg.inv(H)).dot(K))**2 
    res += mu*np.linalg.norm(H,1)
    res += eta*np.linalg.norm(T,1)
    return res

In [308]:
def fixed_interlinks_graphical_lasso(X, K, mu=0.01, eta=0.01, rho=1., 
        tol=1e-3, rtol=1e-5, max_iter=100, verbose=False, return_n_iter=True,
        return_history=False, compute_objective=False, compute_emp_cov=False,
        random_state=None):
    
    random_state = check_random_state(random_state)
    if compute_emp_cov:
        n = X.shape[0] 
        emp_cov = empirical_covariance(X, assume_centered=False)
    else:
        emp_cov = X

    R = emp_cov.copy()
    H = np.random.rand(K.shape[0], K.shape[0])
    H = H.T.dot(H)
    T = emp_cov.copy()
    U = np.zeros((K.shape[1], K.shape[1]))
    
    checks = []
    for iteration_ in range(max_iter):
        R_old = R.copy()
        
        # R update
        M = T - U - K.T.dot(np.linalg.inv(H)).dot(K)
        M = M.T.dot(M)/2
        R = prox_logdet(emp_cov - rho*M, 1/rho)
       # print("----------------------R---------------------\n", R)
        # T update
        M = - R - U - K.T.dot(np.linalg.inv(H)).dot(K)
        
        T = soft_thresholding_sign(M, eta/rho)
        #print("----------------------T---------------------\n",T)
        # H update
        H = _upgrade_H(R, T, K, U, rho, mu)
        #print("----------------------H---------------------\n",H)
        # U update
        KHK = np.linalg.multi_dot((K.T, np.linalg.inv(H), K))
        U += R - T + KHK

        # diagnostics, reporting, termination checks
        
        obj = objective(emp_cov, K, R, T, H, mu, eta, rho) \
            if compute_objective else np.nan
        rnorm = np.linalg.norm(R - T+ KHK)
        snorm = rho *np.linalg.norm(R - R_old)
        check = convergence(
            obj=obj, rnorm=rnorm, snorm=snorm,
            e_pri=(np.sqrt(R.size) * tol + rtol *
                   max(np.sqrt(np.linalg.norm(R)**2 + np.linalg.norm(U)**2),
                       np.linalg.norm(T - KHK))),
            e_dual=(np.sqrt(R.size) * tol + rtol * rho *
                    np.linalg.norm(U))
        )

        if verbose:
            print("obj: %.4f, rnorm: %.4f, snorm: %.4f,"
                  "eps_pri: %.4f, eps_dual: %.4f" % check)

        checks.append(check)
        if check.rnorm <= check.e_pri and check.snorm <= check.e_dual:
            break
        rho_new = update_rho(rho, rnorm, snorm, iteration=iteration_)
        # scaled dual variables should be also rescaled
        U *= rho / rho_new
        rho = rho_new
    else:
        warnings.warn("Objective did not converge.")

    return_list = [R, T, H, emp_cov]
    if return_n_iter:
        return_list.append(iteration_)
    if return_history:
        return_list.append(checks)
    return return_list


In [309]:
random_state=0

A = make_sparse_spd_matrix(dim=15, alpha=0.7, random_state=random_state)

T_true = A[5:,5:]
K_true = A[10:,5:,]
H_true = A[0:5,0:5]
T_true.shape, K_true.shape, H_true.shape

per_cov = K_true*0.3
T_obs = T_true - per_cov.T.dot(H_true).dot(per_cov)

samples = np.random.multivariate_normal(np.zeros(10), np.linalg.inv(T_obs), 50)

In [310]:
fixed_interlinks_graphical_lasso(samples, per_cov, mu=1, eta=1, rho=1., 
        verbose=1, compute_objective=1, compute_emp_cov=1,
        random_state=0)

obj: 111249.7372, rnorm: 470.5490, snorm: 223.0454,eps_pri: 0.0152, eps_dual: 0.0147
obj: 131231559221.1671, rnorm: 512310.6378, snorm: 255507.8219,eps_pri: 5.7401, eps_dual: 5.1378
did not find lambda
obj: 175167485987206492848128.0000, rnorm: 591891013594.0125, snorm: 295944994494.0961,eps_pri: 6617546.1573, eps_dual: 5918915.2736


LinAlgError: Singular matrix